# data

> Fill in a module description here

In [ ]:
#| default_exp data

In [ ]:
#| export
from typing import (
    Iterable,
    Callable,
    Iterator,
    Optional,
    List
)
import minima as mi
from minima import Tensor
from minima import init
import fastcore.all as fc
import random

In [ ]:
#| export
class Sampler:
    """
    A custom sampler class.

    Args:
        ds (Iterable[int]): Iterable of indices.
        shuffle (bool): Whether to shuffle the indices.

    Example:
        >>> x = range(10)
        >>> sampler = Sampler(x, shuffle=True)
    """

    def __init__(self, ds: Iterable[int], shuffle: bool = False):
        self.n = len(ds)
        self.shuffle = shuffle

    def __iter__(self) -> Iterator[int]:
        res = list(range(self.n))
        if self.shuffle: random.shuffle(res)
        return iter(res)

In [ ]:
#| export
class BatchSampler:
    """
    A custom batch sampler class.

    Args:
        sampler (Sampler): The sampler to use.
        bs (int): Batch size.
        drop_last (bool): Whether to drop the last batch if it is smaller than the batch size.

    Example:
        >>> x = range(10)
        >>> sampler = Sampler(x, shuffle=True)
        >>> batch_sampler = BatchSampler(sampler, bs=3)
    """

    def __init__(self, sampler: Sampler, bs: int, drop_last: bool = False):
        self.sampler = sampler
        self.bs = bs
        self.drop_last = drop_last

    def __iter__(self):
        yield from fc.chunked(iter(self.sampler), self.bs, drop_last=self.drop_last)


In [ ]:
#| export
class Dataset():
    r"""An abstract class representing a :class:`Dataset`.

    All datasets that represent a map from keys to data samples should subclass
    it. All subclasses should overwrite:
    `__getitem__`, supporting fetching a data sample for a given key.
    `__len__`, which is expected to return the size of the dataset.
    """

    def __init__(self, transforms: Optional[List]=None):
        self.transforms = transforms

    def __getitem__(self, index) -> object:
        """
        Get an item from the dataset at the given index.

        Args:
            i (int): Index of the item.

        Returns:
            Tuple[float, float]: A tuple containing the input data and target label at the given index.

        Example:
            >>> dataset[0]
            (1, 0)
        """
        
        raise NotImplementedError

    def __len__(self) -> int:
        """
        Get the length of the dataset.

        Returns:
            int: Length of the dataset.

        Example:
            >>> len(dataset)
            5
        """
        
        raise NotImplementedError

    def apply_transforms(self, x):
        if self.transforms is not None:
            for tfms in self.transforms:
                x = tfms(x)
        return x

In [ ]:
#|export


def collate(b):
    xs,ys = zip(*b)
    return torch.stack(xs),torch.stack(ys)

class DataLoader:
    """
    A custom data loader class.

    Args:
        ds (Dataset): The dataset to load.
        bs (int): Batch size.

    Example:
        >>> dataloader = DataLoader(dataset, batch_size)
    """

    def __init__(self,
                 dataset: Dataset,
                 batch_size: int = 1,
                 shuffle: bool = True,
                 sampler: Sampler = None,
                 batch_sampler: BatchSampler = None,
                 num_workers: int = 0,
                 collate_fn: callable = None,
                 drop_last: bool = False):

        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.sampler = sampler if sampler else Sampler(dataset, shuffle)
        self.batch_sampler = batch_sampler if batch_sampler else BatchSampler(self.sampler, batch_size, drop_last)
        self.num_workers = num_workers # --> TODO: implement a multiprocessing DataLoader :3
        self.collate_fn = collate
        self.drop_last = drop_last

    def __iter__(self):
        """
        Get an iterator over the DataLoader.

        Yields:
            Tuple[float, float]: A tuple containing a batch of input data and target labels.

        Example:
            >>> for batch in dataloader:
            >>>     # Process the batch
        """
        if self.num_workers:
            with mp.Pool(self.num_workers) as ex:
                yield from ex.map(self.dataset.__getitem__,  iter(self.batch_sampler))
        else:
            yield from (self.dataset[batch_idxs] for batch_idxs in self.batch_sampler)



In [ ]:
X = init.rand(100, 10)
Y = init.randb(X.shape[0])
X.shape, Y.shape

((100, 10), (100,))

In [ ]:
class MiDataset(Dataset):

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self) -> int:
        return self.x.shape[0]

    def __getitem__(self, i: int):
        return self.x[i], self.y[i]

In [ ]:
ds = MiDataset(X,Y)

In [ ]:
len(ds)

100

In [ ]:
ds[:10]

(minima.Tensor(
 [[0.835655 0.355682 0.341312 0.877118 0.964982 0.161331 0.871119 0.653924 0.112947 0.995913]
  [0.5362   0.935408 0.452379 0.046165 0.030116 0.169965 0.570334 0.426115 0.03146  0.132633]
  [0.163104 0.91955  0.635856 0.995347 0.993711 0.460345 0.894598 0.273158 0.124523 0.119298]
  [0.638094 0.598965 0.745832 0.370086 0.670154 0.071052 0.32124  0.806154 0.743922 0.123329]
  [0.946646 0.726528 0.724754 0.568195 0.282958 0.240825 0.941848 0.742167 0.750217 0.275818]
  [0.513921 0.476381 0.050706 0.543432 0.484169 0.318996 0.580121 0.343467 0.814567 0.417969]
  [0.753226 0.521739 0.940589 0.336658 0.906802 0.777629 0.836282 0.378091 0.259586 0.798654]
  [0.341872 0.972311 0.902511 0.193453 0.089093 0.564276 0.213413 0.927187 0.968153 0.307819]
  [0.749471 0.773822 0.469319 0.862515 0.167192 0.390072 0.942944 0.92225  0.460263 0.55879 ]
  [0.925903 0.233595 0.177896 0.976798 0.043768 0.61924  0.939949 0.875439 0.068964 0.574139]]),
 minima.Tensor(
 [False  True  True  True

## Export

In [ ]:
import nbdev; nbdev.nbdev_export()